## Gráficos interactivos ("widgets")
Veamos cómo se puede hacer este tipo de trabajo en forma interactiva. Para ello **Matplotlib** tiene un submódulo `widgets` con rutinas que están diseñadas para funcionar con cualquier *backend* interactivo. (más información en: http://matplotlib.org/api/widgets_api.html)

### Cursor

Empecemos estudiando como agregar un indicador de la posición del cursor a un gráfico.

```python

# Archivo: ejemplo_cursor.py

from matplotlib.widgets import Cursor
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8,6))

x, y = 4 * (np.random.rand(2, 100) - .5)
ax.plot(x, y, 'o')
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)

# Usamos: useblit = True en el backed gtkagg
cursor = Cursor(ax, useblit=True, color='red', linewidth=2)

plt.show()
```

En este caso, el programa está escrito separadamente y lo podemos ejecutar desde la notebook (como en este caso) o desde una terminal independientemente.
Para ejecutar un script en forma interactiva desde la *notebook* de *Jupyter* debemos setear el *backend* a una opción interactiva (que no es el valor por default en las notebooks). En este caso vamos a usar el backend *tk* 

In [ ]:
import numpy as np
%matplotlib ipympl
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

x, y = 4 * (np.random.rand(2, 100) - .5)
ax.plot(x, y, 'o')
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)

In [ ]:
%matplotlib tk
import matplotlib.pyplot as plt
%run scripts/ejemplo_cursor.py

In [ ]:
from matplotlib.widgets import Cursor

# Usamos: useblit = True en el backed gtkagg
cursor = Cursor(ax, useblit=True, color='red', linewidth=2)

plt.show()

En este ejemplo simple conocemos casi todas las líneas (creamos la figura y graficamos). Las líneas novedosas y relevantes son

1. La primera línea importando la función `Cursor()` para describir el *cursor* o *mouse*:
```python
from matplotlib.widgets import Cursor
```
2. La línea en que usamos la función `Cursor`
  ```python
  cursor = Cursor(ax, useblit=True, color='red', linewidth=2)
  ```
  que crea el objeto `Cursor`. La forma de esta función es:
  
  ```python
  Cursor(ax, horizOn=True, vertOn=True, useblit=False, **lineprops)
  ```
  y toma como argumento el eje en el cuál agregamos el cursor. Como argumentos opcionales se puede controlar la visibilidad de la línea horizontal (`horizOn`) y vertical (`vertOn`) que pueden tomar valores lógicos `True` o `False`. Además tiene argumentos *keyword* para controlar la apariencia de las líneas. En este ejemplo pusimos explícitamente que queremos una línea roja con un grosor igual a `2`.

### Manejo de eventos

Para que la interactividad sea útil es importante obtener datos de nuestra interacción con el gráfico. Esto se obtiene con lo que se llama "manejo de eventos" ("Event handling").

Para recibir *events* necesitamos **escribir y conectar** una función que se activa cuando ocurre el evento (*callback*). Veamos un ejemplo simple pero importante, donde imprimimos las coordenadas donde se presiona el *mouse*.

```python 
# Archivo: ejemplo_callback.py
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(np.random.rand(10), 'o-', lw=3)

def onclick(event):
  print('button=%d, x=%d, y=%d, xdata=%f, ydata=%f' % (event.button, event.x, event.y, event.xdata, event.ydata))

cid = fig.canvas.mpl_connect('button_press_event', onclick)
plt.show()
```

![](figuras/uso_callbacks1.png)

En este ejemplo utilizamos el método `mpl_connect` del objeto `canvas`.

- El objeto `canvas` es el área donde se dibuja la figura. 
- La función `mpl_connect` realiza la conección de la función (que aquí llamamos `onclick`) con la figura.
  Esta función toma como argumento el *event* (que, para nosotros, es interpretado automaticamente por Matplotlib)

- El objeto `event` es de tipo `button_press_event`. Se dispara cuando apretamos un botón del *mouse* y *matplotlib* le pasa como argumento un objeto del tipo `event` que contiene información. Nosotros estamos imprimiendo la siguiente información que contiene `event` por pantalla:
  - `event.button`: indica que botón se presionó
  - `event.x`, `event.y`: dan la información sobre el índice en los ejes horizontal y vertical
  - `event.xdata, event.ydata`: dan los valores de los datos en los ejes.


Puede leer más información sobre el manejo de eventos en http://matplotlib.org/users/event_handling.html.

### Ejemplos integrados

Continuando con esta idea vamos a usar la capacidad de poder interactuar con el gráfico para elegir una zona del gráfico de la cuál obtenemos información sobre los datos.

```python
# Archivo: scripts/analizar_figura_1.py

import matplotlib.pyplot as plt
from matplotlib.widgets import Cursor

img = plt.imread('../figuras/imagen_flujo_gray.jpg')
ymax = img.max()

def seleccionar(event):
  """Secuencia:
  1. Encuentro el punto donde el mouse hizo 'click'
  2. Le doy valores a la línea vertical
  3. Le doy valores a la curva en el grafico de la derecha
  4. y 5. Grafico los nuevos valores
  """
  x0 = event.xdata
  n0 = int(x0)
  l1.set_data([[n0, n0], [0., 1.]])
  l2.set_data(range(img.shape[0]), img[:, n0])
  l1.figure.canvas.draw()
  l2.figure.canvas.draw()


# Defino la figura
fig, (ax1, ax2) = plt.subplots(figsize=(12, 4), ncols=2)

# Mostramos la imagen como un mapa de grises
ax1.imshow(img, cmap='gray', interpolation='nearest')
ax1.axis('off')

# Agrego la línea inicial en un valor inicial
x0 = 100
l1 = ax1.axvline(x0, color='r', ls='--', lw=3)

# Grafico de la derecha
l2, = ax2.plot(img[:, x0], 'r-', lw=2, label='corte')
ax2.set_ylim((0, ymax))
ax2.set_xlabel(u'posición en eje $y$ (pixels)')
ax2.set_ylabel('Intensidad')
ax2.legend(loc='best')

fig.tight_layout()

# Agrego el cursor y conecto la accion de presionar a la funcion click
cursor = Cursor(ax1, horizOn=False, vertOn=True, useblit=True,
                color='blue', linewidth=1)
fig.canvas.mpl_connect('button_press_event', seleccionar)

plt.show()

```


In [7]:
!pwd
%matplotlib tk
%run ../scripts/analizar_figura.py

/Users/flavioc/Library/Mobile Documents/com~apple~CloudDocs/Documents/cursos/Python/intro-python-IB/clases


FileNotFoundError: [Errno 2] No such file or directory: '../figuras/imagen_flujo_gray.jpg'

Este es un ejemplo un poco más largo (y un poquito más complejo).

1. Importamos los módulos y funciones necesarias.
  - `matplotlib.pyplot as plt` para casi todo
  - `imageio` para leer la figura
  - `from matplotlib.widgets import Cursor` para importar el objeto `Cursor` que nos muestra la posición del mouse.
2. Leemos la imagen de archivo, creamos la figura y la mostramos.
3. Elegimos un valor de $x$ inicial (igual a 100) y agregamos una línea vertical en ese punto.
4. Creamos la figura de la derecha con los datos tomados de la columna correspondiente de la matriz que representa la imagen.
6. Agregamos *labels* y ajustamos las distancias
5. Mostramos el cursor y le conectamos el evento (standard) `button_press_event` a nuestra función `seleccionar()`.
6. La función `seleccionar()` toma como argumento el evento que se dispara por interacción con el usuario.  
El argumento `event` lo pasa automáticamente *Matplotlib*. En este caso es un *click* del mouse en una zona del gráfico.

La función `seleccionar(event)`:
1. Del argumento `event` extrae la posición en el eje horizontal y lo asigna a la variable `x0`.
2. El índice en el eje horizontal (`n0`).
2. Actualiza los datos de la línea `l1` con valores para la línea vertical en el panel izquierdo.
4. Actualiza la línea `l2` en la figura de la derecha con el corte en `x0`.
5. Actualiza el dibujo de las líneas sobre el *canvas*.

El siguiente ejemplo es muy similar al anterior. Sólo estamos actualizando la leyenda, para tener información del punto seleccionado.

In [ ]:
%run scripts/analizar_figura_2.py

```python
# Archivo:analizar_figura_2.py
import matplotlib.pyplot as plt
from scipy import misc
from matplotlib.widgets import Cursor

img = plt.imread('../figuras/imagen_flujo_gray.jpg')
ymax = img.max()


def click(event):
  """Secuencia:
  1. Encuentro el punto donde el mouse hizo 'click'
  2. Le doy valores a la línea vertical
  3. Le doy valores a la curva en el grafico de la derecha
  4. y 5. Grafico los nuevos valores
  """
  x0 = event.xdata
  n0 = int(x0)
  l1.set_data([[n0, n0], [0., 1.]])
  l2.set_data(range(img.shape[0]), img[:, n0])
  leg2.texts[0].set_text('corte en {:.1f}'.format(x0))
  l1.figure.canvas.draw()
  l2.figure.canvas.draw()


# Defino la figura
# Defino la figura
fig, (ax1, ax2) = plt.subplots(figsize=(12, 4), ncols=2)

ax1.imshow(img, cmap="gray", interpolation='nearest')
ax1.axis('off')
# Agrego la línea inicial en un valor inicial
x0 = 100
l1 = ax1.axvline(x0, color='r', ls='--', lw=3)

# Grafico de la derecha
l2, = ax2.plot(img[:, x0], 'r-', lw=2, label='corte en {:.1f}'.format(x0))
ax2.set_ylim((0, ymax))
ax2.set_xlabel(u'posición en eje $y$ (pixels)')
ax2.set_ylabel('Intensidad')
leg2 = ax2.legend(loc='best')

fig.tight_layout()

# Agrego el cursor y conecto la accion de presionar a la funcion click
cursor = Cursor(ax1, horizOn=False, vertOn=True, useblit=True,
                color='blue', linewidth=1)
fig.canvas.mpl_connect('button_press_event', click)

plt.show()
```

Las diferencias más notables con el ejemplo anterior son:

1. Al crear la leyenda, asignamos el objeto creado a la variable `leg2`, en la línea:
  ```python
  leg2 = ax2.legend(loc='best')
  ```
2. En la función `click(event)` (equivalente a `seleccionar(evento)` en el ejemplo anterior) actualizamos el texto de la leyenda con el valor de `x0`:
  ```python
  leg2.texts[0].set_text('corte en {:.1f}'.format(x0))
  ```

-----

## Ejercicios 14 (c)

4. Modificar el ejemplo anterior (**analizar_figura_2.py**) para presentar tres gráficos, agregando a la izquierda un panel donde se muestre el corte horizontal de la misma manera que en el ejercicio anterior. Al seleccionar con el *mouse* debe mostrar los dos cortes (horizontal y vertical).

-----

.